# 2.5 Automatic Differentiation
Differentiation is a crucial step in nearly all deep learning optimization algorithms. Automatic differentiation enables the system to subsequently backpropagate gradients.

## 2.5.1 A Simple Example

In [6]:
import torch

x = torch.arange(4.0)
print(x)
x.requires_grad_(True)  # Same as `x = torch.arange(4.0, requires_grad=True)`
print(x.grad)  # The default value is None

y = 2 * torch.dot(x, x)
print(y)
y.backward()
print(x.grad)
print(x.grad == 4 * x)

# PyTorch accumulates the gradient in default, we need to clear the previous values
x.grad.zero_()
y = x.sum()
print(y)
y.backward()
print(x.grad)

tensor([0., 1., 2., 3.])
None
tensor(28., grad_fn=<MulBackward0>)
tensor([ 0.,  4.,  8., 12.])
tensor([True, True, True, True])
tensor(6., grad_fn=<SumBackward0>)
tensor([1., 1., 1., 1.])


## 2.5.2 Backward for Non-Scalar Variables
Technically, when y is not a scalar, the most natural interpretation of the differentiation of a vector y with respect to a vector x is a matrix. For higher-order and higher-dimensional y and x, the differentiation result could be a high-order tensor.

In [8]:
# Invoking `backward` on a non-scalar requires passing in a `gradient` argument
# which specifies the gradient of the differentiated function w.r.t `self`.
# In our case, we simply want to sum the partial derivatives, so passing
# in a gradient of ones is appropriate
x.grad.zero_()
y = x * x
print(y)
# y.backward(torch.ones(len(x))) equivalent to the below
y.sum().backward()
print(x.grad)

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)
tensor([0., 2., 4., 6.])


## 2.5.3 Detaching Computation

In [14]:
x.grad.zero_()
y = x * x
print(y)
u = y.detach()
print(u)
z = u * x
print(z)
z.sum().backward()
x.grad == u

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)
tensor([0., 1., 4., 9.])
tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>)


tensor([True, True, True, True])

In [15]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## 2.5.4 Computing the Gradient of Python Control Flow

In [22]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
print(a)

d = f(a)
print(d)

d.backward()
print(a.grad == d / a)

tensor(1.2038, requires_grad=True)
tensor(1232.6451, grad_fn=<MulBackward0>)
tensor(True)
